This is a jupyter notebook to properly upload the chosen model for this assignment

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
import joblib

In [3]:
class DateToDayOfYear(BaseEstimator, TransformerMixin):
    def __init__(self):

        self.month_map = {
            'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
            'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
        }

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        X['month_num'] = X['month'].map(self.month_map)

        def get_day_of_year(row):
            try:
                return datetime(2023, int(row['month_num']), int(row['day'])).timetuple().tm_yday ## We select 2023 because it has 365 days
            except:
                return -1

        return X.apply(get_day_of_year, axis=1).values.reshape(-1, 1)
    

def cyclical_transform(X, period):
    X = np.array(X, dtype=float)
    sin_trans = np.sin(2 * np.pi * X / period)
    cos_trans = np.cos(2 * np.pi * X / period)
    return np.c_[sin_trans, cos_trans]

def cyclical_transform_365(X):
    return cyclical_transform(X, period=365)

def identity_transform(X):
    return np.asarray(X)


In [4]:
class PdaysTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, missing_value=-1):
        self.missing_value = missing_value
        self.max_int64_ = np.iinfo(np.int64).max

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = np.asarray(X, dtype=np.float64) 
        X = X.copy()
        X[X == self.missing_value] = self.max_int64_
        return X


In [5]:
model = joblib.load("final_model_catboost_pipeline.joblib")

X_new = pd.read_pickle("bank_competition.pkl")
if "deposit" in X_new.columns:
    Y_new = X_new["deposit"]
    X_new = X_new.drop(columns=["deposit"])
    

preds = model.predict(X_new)
preds

array([0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0])